In [1]:
# Import all necessary libraries
import torch
import os
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model

print("All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")



All libraries imported successfully!
PyTorch version: 2.7.0+cu118
Transformers version: 4.53.0
CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
# SFT Configuration variables
# Start with a small and fast model for the first run
# This can be swapped later for a larger model like 'gpt2-medium' or 'microsoft/DialoGPT-medium'
base_model_name = 'distilgpt2'

# Dataset and output paths
dataset_path = './data/train_prefs.jsonl'
sft_output_dir = './models/sft'

print(f"Configuration:")
print(f"Base model: {base_model_name}")
print(f"Dataset path: {dataset_path}")
print(f"SFT output directory: {sft_output_dir}")

# Ensure output directory exists
os.makedirs(sft_output_dir, exist_ok=True)


Configuration:
Base model: distilgpt2
Dataset path: ./data/train_prefs.jsonl
SFT output directory: ./models/sft


In [3]:
# Load and format the dataset for SFT training (TRL 0.19.0 compatible)
print("Loading dataset for SFT...")

# Load the prepared dataset
train_dataset = load_dataset('json', data_files=dataset_path)['train']

print(f"Dataset loaded with {len(train_dataset)} examples")
print(f"Original keys: {list(train_dataset[0].keys())}")

def format_example(example):
    """
    Format example for SFT training with TRL 0.19.0 compatible format.
    TRL 0.19.0 expects separate 'prompt' and 'completion' fields.
    """
    # Format prompt with instruction template
    prompt = f"### Human:\n{example['prompt']}\n\n### Assistant:\n"
    completion = example['chosen']
    
    return {
        "prompt": prompt,
        "completion": completion
    }

# Apply formatting to the entire dataset
print("\nFormatting dataset for TRL 0.19.0...")
formatted_dataset = train_dataset.map(format_example)

print(f"Formatted dataset keys: {list(formatted_dataset[0].keys())}")
print(f"\nExample formatted prompt (first 200 chars):")
print(formatted_dataset[0]['prompt'][:200] + "...")
print(f"\nExample completion (first 200 chars):")
print(formatted_dataset[0]['completion'][:200] + "...")

print(f"\nDataset ready for SFT training with {len(formatted_dataset)} examples!")


Loading dataset for SFT...


Generating train split: 0 examples [00:00, ? examples/s]

Dataset loaded with 5000 examples
Original keys: ['chosen', 'rejected', 'prompt']

Formatting dataset for TRL 0.19.0...


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Formatted dataset keys: ['chosen', 'rejected', 'prompt', 'completion']

Example formatted prompt (first 200 chars):
### Human:
Human: Why did cells originally combine together to create life?

### Assistant:
...

Example completion (first 200 chars):
Because their simple components -- chemicals -- interacted in particular ways.  And because of chemical processes involving acids and bases, certain kinds of chemicals can begin to self-organize into ...

Dataset ready for SFT training with 5000 examples!


In [4]:
# Load base model and tokenizer
print(f"Loading tokenizer and model: {base_model_name}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Critical step: Check and set pad token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("✓ Pad token set to EOS token")
else:
    print("✓ Pad token already exists")

print(f"Tokenizer loaded - Vocab size: {len(tokenizer)}")

# Load the base model
print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,  # Use bfloat16 for better training stability and RTX 4060 compatibility
    device_map="auto"  # Automatically place model on available GPU
)

print(f"✓ Model loaded successfully!")
print(f"Model parameters: {model.num_parameters():,}")
print(f"Model device: {next(model.parameters()).device}")

# Print model architecture summary
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


Loading tokenizer and model: distilgpt2
✓ Pad token set to EOS token
Tokenizer loaded - Vocab size: 50257
Loading base model...
✓ Model loaded successfully!
Model parameters: 81,912,576
Model device: cuda:0
Total parameters: 81,912,576
Trainable parameters: 81,912,576


In [5]:
# Configure LoRA (Parameter-Efficient Fine-Tuning)
print("Configuring LoRA for efficient fine-tuning...")

# Create LoRA configuration
lora_config = LoraConfig(
    r=16,                    # Rank of the update matrices (higher = more parameters)
    lora_alpha=32,           # LoRA scaling factor (usually 2*r)
    lora_dropout=0.05,       # Dropout rate for LoRA layers
    bias="none",             # No bias updates
    task_type="CAUSAL_LM"    # Task type for causal language modeling
)

print("LoRA Configuration:")
print(f"  Rank (r): {lora_config.r}")
print(f"  Alpha: {lora_config.lora_alpha}")
print(f"  Dropout: {lora_config.lora_dropout}")
print(f"  Bias: {lora_config.bias}")
print(f"  Task type: {lora_config.task_type}")

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Print trainable parameters after applying LoRA
model.print_trainable_parameters()

print("✓ LoRA configuration complete!")


Configuring LoRA for efficient fine-tuning...
LoRA Configuration:
  Rank (r): 16
  Alpha: 32
  Dropout: 0.05
  Bias: none
  Task type: CAUSAL_LM
trainable params: 294,912 || all params: 82,207,488 || trainable%: 0.3587
✓ LoRA configuration complete!


d:\downloads\python\Lib\site-packages\peft\tuners\lora\layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [6]:
# Configure training arguments optimized for RTX 4060
print("Configuring training arguments for RTX 4060...")

training_args = TrainingArguments(
    output_dir=sft_output_dir,
    per_device_train_batch_size=1,        # Very small batch size for 8GB VRAM
    gradient_accumulation_steps=8,         # Effective batch size = 1*8 = 8
    learning_rate=2e-4,                    # Learning rate for LoRA
    logging_steps=20,                      # Log every 20 steps
    num_train_epochs=1,                    # Number of training epochs
    max_steps=-1,                          # Train for epochs, not steps
    bf16=True,                             # Essential for RTX 4060 performance
    save_strategy="epoch",                 # Save at the end of each epoch
    eval_strategy="no",                    # Fixed: Changed from evaluation_strategy to eval_strategy
    remove_unused_columns=False,           # Keep all dataset columns
    push_to_hub=False,                     # Don't push to Hugging Face Hub
    report_to=None,                        # Disable wandb/tensorboard logging
    dataloader_pin_memory=False,           # Reduce memory usage
    gradient_checkpointing=True,           # Trade compute for memory
    warmup_steps=10,                       # Add warmup steps
    weight_decay=0.01,                     # Small weight decay
)

print("Training Arguments:")
print(f"  Batch size per device: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation steps: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Number of epochs: {training_args.num_train_epochs}")
print(f"  BF16 enabled: {training_args.bf16}")

# Convert dataset back to simple text format for compatibility
print("\nConverting dataset format for stable training...")

def convert_to_text_format(example):
    """Convert to simple text format that works reliably"""
    text = f"Human: {example['prompt']}\n\nAssistant: {example['chosen']}"
    return {"text": text}

# Apply conversion and remove original columns
text_dataset = formatted_dataset.map(convert_to_text_format, 
                                    remove_columns=formatted_dataset.column_names)

print(f"Text dataset ready with {len(text_dataset)} examples")
print(f"Sample text (first 200 chars): {text_dataset[0]['text'][:200]}...")

# Instantiate the SFTTrainer with corrected configuration
print("\nInitializing SFTTrainer with corrected parameters...")

try:
    # 完全重置Accelerator状态
    import torch
    torch.cuda.empty_cache()
    
    from accelerate import Accelerator
    from accelerate.state import AcceleratorState
    
    # 清除共享状态
    if hasattr(AcceleratorState, '_shared_state') and AcceleratorState._shared_state:
        AcceleratorState._shared_state.clear()
    
    # 创建新的加速器实例
    accelerator = Accelerator()
    
    # 正确的SFTTrainer初始化（移除所有不支持的参数）
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=text_dataset,
        # 只保留基本参数，移除所有可能导致问题的参数
    )
    
    print("✅ SFTTrainer initialized successfully!")
    
except Exception as e:
    print(f"❌ SFTTrainer initialization failed: {e}")
    
    # Try with even more minimal configuration
    print("Trying with minimal TRL-compatible configuration...")
    try:
        trainer = SFTTrainer(
            model=model,
            args=training_args,
            train_dataset=text_dataset,
        )
        print("✅ SFTTrainer initialized with minimal config!")
    except Exception as e2:
        print(f"❌ Minimal config also failed: {e2}")
        print("The issue might be with TRL version compatibility.")

# Calculate training steps
total_steps = len(text_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs
print(f"Total training steps: {total_steps}")
print(f"Estimated training time: {total_steps * 3} seconds (approximate)")


Configuring training arguments for RTX 4060...
Training Arguments:
  Batch size per device: 1
  Gradient accumulation steps: 8
  Effective batch size: 8
  Learning rate: 0.0002
  Number of epochs: 1
  BF16 enabled: True

Converting dataset format for stable training...


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Text dataset ready with 5000 examples
Sample text (first 200 chars): Human: ### Human:
Human: Why did cells originally combine together to create life?

### Assistant:


Assistant: Because their simple components -- chemicals -- interacted in particular ways.  And beca...

Initializing SFTTrainer with corrected parameters...


Adding EOS to train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1875 > 1024). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


✅ SFTTrainer initialized successfully!
Total training steps: 625
Estimated training time: 1875 seconds (approximate)


In [7]:
# Start training and save the model
print("🚀 Starting SFT training...")
print("This may take some time depending on your dataset size and hardware.")
print("You can monitor the progress through the logging output below.")
print("-" * 60)

# Reset accelerator state to fix potential state issues
print("Resetting accelerator state for clean training...")
try:
    from accelerate import Accelerator
    # Create a new accelerator instance to reset state
    accelerator = Accelerator()
    print("✅ Accelerator state reset successfully")
except Exception as e:
    print(f"⚠️ Could not reset accelerator: {e}")
    print("Continuing without accelerator reset...")

# Pre-training verification
print("Pre-training verification:")
print(f"Model device: {next(model.parameters()).device}")
print(f"Model dtype: {next(model.parameters()).dtype}")
print(f"Has trainable parameters: {any(p.requires_grad for p in model.parameters())}")

# Ensure model is in training mode
model.train()

# Start the training process with comprehensive error handling
try:
    print("Attempting to start training...")
    training_output = trainer.train()
    print("-" * 60)
    print("🎉 SFT training finished successfully!")
    
    # Display training statistics if available
    if hasattr(training_output, 'training_loss'):
        print(f"Final training loss: {training_output.training_loss:.4f}")
    
except RuntimeError as e:
    error_msg = str(e).lower()
    if "does not require grad" in error_msg:
        print("❌ Gradient flow error detected.")
        print("🔧 Trying to fix gradient requirements...")
        
        # Try to enable gradients for LoRA parameters
        for name, param in model.named_parameters():
            if 'lora' in name.lower() or param.requires_grad:
                param.requires_grad_(True)
        
        print("Attempting training again with fixed gradients...")
        try:
            training_output = trainer.train()
            print("✅ Training successful after gradient fix!")
        except Exception as e2:
            print(f"❌ Training still failed: {e2}")
            
    elif "acceleratorstate" in error_msg or "distributed_type" in error_msg:
        print("❌ Accelerator state error detected.")
        print("🔧 Suggested solutions:")
        print("1. Restart the notebook kernel")
        print("2. Try: pip install --upgrade accelerate transformers")
        print("3. This is a known issue with certain TRL/Accelerate versions")
        
    else:
        print(f"❌ Training failed with error: {e}")
        print("🔧 Suggested solutions:")
        print("1. Try reducing batch size or sequence length")
        print("2. Check GPU memory availability")
        print("3. Consider using different TRL version")
    
except Exception as e:
    print(f"❌ Unexpected training error: {e}")
    print("The training encountered an unexpected issue.")

# Try to save the LoRA adapters regardless
print("\nAttempting to save the trained model...")
try:
    trainer.save_model()
    print(f"✅ Model saved to: {sft_output_dir}")
    
    # Verify saved files
    if os.path.exists(sft_output_dir):
        saved_files = os.listdir(sft_output_dir)
        if saved_files:
            print(f"\nFiles in {sft_output_dir}:")
            total_size = 0
            for file in saved_files:
                file_path = os.path.join(sft_output_dir, file)
                if os.path.isfile(file_path):
                    file_size = os.path.getsize(file_path) / (1024 * 1024)  # Size in MB
                    total_size += file_size
                    print(f"  📄 {file} ({file_size:.2f} MB)")
            
            print(f"\n🎯 Total size: {total_size:.2f} MB")
            
            if 'adapter_model.safetensors' in saved_files:
                print("✅ LoRA adapters saved successfully!")
                print("These adapters can be used for inference or further training.")
            else:
                print("⚠️  adapter_model.safetensors not found - training may not have completed")
        else:
            print("❌ No files found in output directory")
    else:
        print("❌ Output directory does not exist")
        
except Exception as e:
    print(f"❌ Could not save model: {e}")

print(f"\n📝 Summary:")
print(f"Base model: {base_model_name}")
print(f"Training attempted on {len(text_dataset)} examples")
print(f"Output directory: {sft_output_dir}")
print("\n🔄 Training process completed (check messages above for success/failure status)")


🚀 Starting SFT training...
This may take some time depending on your dataset size and hardware.
You can monitor the progress through the logging output below.
------------------------------------------------------------
Resetting accelerator state for clean training...
✅ Accelerator state reset successfully
Pre-training verification:
Model device: cuda:0
Model dtype: torch.bfloat16
Has trainable parameters: True
Attempting to start training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
d:\downloads\python\Lib\site-packages\torch\utils\checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


❌ Gradient flow error detected.
🔧 Trying to fix gradient requirements...
Attempting training again with fixed gradients...
❌ Training still failed: element 0 of tensors does not require grad and does not have a grad_fn

Attempting to save the trained model...
✅ Model saved to: ./models/sft

Files in ./models/sft:
  📄 adapter_config.json (0.00 MB)
  📄 adapter_model.safetensors (1.13 MB)
  📄 merges.txt (0.44 MB)
  📄 README.md (0.00 MB)
  📄 special_tokens_map.json (0.00 MB)
  📄 tokenizer.json (3.39 MB)
  📄 tokenizer_config.json (0.00 MB)
  📄 training_args.bin (0.01 MB)
  📄 vocab.json (0.76 MB)

🎯 Total size: 5.73 MB
✅ LoRA adapters saved successfully!
These adapters can be used for inference or further training.

📝 Summary:
Base model: distilgpt2
Training attempted on 5000 examples
Output directory: ./models/sft

🔄 Training process completed (check messages above for success/failure status)
